# Create a FAISS based Vector Index for DBCopilot with AzureML
We'll walk through setting up an AzureML Pipeline which grounding a Database into a LangChain-compatible FAISS Vector Index and create the Prompt flow to consume this index to serve as a DBCopilot chatbot.

In [25]:
# create an conda environment with the exported file AMLenv.conda which should be in the same project directory.

In [26]:
# If AMLenv.conda is not available, use the following commands to install the required packages in the python/conda environment
# 
# %pip install azureml-core
# %pip install azure-ai-ml
# %pip install -U 'azureml-rag[faiss]>=0.1.11'

In [27]:
# If `import win32file` fails with a DLL error then run the following and restart kernel:
# %pip uninstall -y pywin32
# %conda install -y --force-reinstall pywin32

In [28]:
# System imports
import os
import json

In [29]:
#
# retrieve and print environment variables
#
# Load environment variables from an .env file
# this is a way to keep sensitive information out of the codebase
# you can use the .env.sample file as a template for the .env file
#  - copy the .env.sample file to .env
#  - fill in the values for your environment
#  - make sure .env and it's realative path are in .gitignore
#
# The following code allows for .env file to be in same directory as script
# or you can specify the path relative to the notebook to the .env file
# 
from os.path import join
from dotenv import load_dotenv
dotenv_path = join(os.getcwd(), '.env')
print(dotenv_path)
load_dotenv(dotenv_path)

c:\Users\davidtorres\Code\GitHub-dat\random-samples\AzureML\DBCopilot\.env


True

In [30]:
# Execution Variables

# Tenent and Application Id settings
tenant_id = os.getenv('TENANT_ID')                                      # "<enter the tenent id>"                   << Used for EnvironmentCredential
application_client_id = os.getenv('CLIENT_ID')                          # "<enter application client id>"           << Used for EnvironmentCredential
application_client_secret = os.getenv('CLIENT_SECRET')                  # "<enter application client secret>"       << Used for EnvironmentCredential

# Azure Workspace settings
subscription_id = os.getenv('SUBSCRIPTION_ID')                          # "<enter the subscription id>"             << Used in workspace.json settings file
resource_group_name = os.getenv('RESOURCE_GROUP')                       # "<enter the resource group name>"         << Used in workspace.json settings file
workspace_name = os.getenv('WORKSPACE_NAME')                            # "<enter the azure ml workspace name>"     << Used in workspace.json settings file
default_compute=os.getenv('CLUSTER_NAME')                               # "<enter dedicate compute cluster name>"   << !! Only works with dedicate compute cluster
                                                                        # "serverless"                              << !! "serverless" and "named" compute instances are 
                                                                        #  NOTE >>>>                                << !! currently causing failure in generate_meta_embedding step

# Azure OpenAI settings
aoai_connection_name = "Default_AzureOpenAI"                            # "<default is Default_AzureOpenAI>"        << Use the Azure OpenAI resource connection name
aoai_embedding_model_name = "text-embedding-ada-002"                    # "<default to text-embedding-ada-002"      << Recommendation is text-embedding-ada-002
aoai_completion_model_name = "gpt-35-turbo"                             # "<default to gpt-35-turbo (0301)>"        << Recommendation is gpt-35-turbo (0301)
                                                                        # NOTE >>>>                                 << !! gpt-35-turbo (0613) and gpt-35-turbo-16 are not support for completions

# Set vector index asset name
datastore_name = os.getenv('DATA_STORE_NAME')                           # "<enter the name of the Datastore>"       << The database registered in Data > Datastore to create embeddings
datastore_scope = "array"                                               # "<enter the scope to be indexed>"         << "all" = all tables, anything else need a list of tables/views 
data_asset_name = f"{datastore_name}_{datastore_scope}_llm_index"       # "<enter the vector index suffix>"         << The index to be created in Data > Data Asset

If datastore_scope is not "all" then create a list of tables to be in the scope of the vector index

In [31]:
if datastore_scope != "all":
    tables = []
    tables.append('[HumanResources].[Department]')
    tables.append('[HumanResources].[Employee]')
    tables.append('[HumanResources].[EmployeeDepartmentHistory]')
    tables.append('[HumanResources].[EmployeePayHistory]')
    tables.append('[HumanResources].[JobCandidate]')
    tables.append('[HumanResources].[Shift]')
    tables.append('[Person].[Address]')
    tables.append('[Person].[AddressType]')
    tables.append('[Person].[BusinessEntity]')
    tables.append('[Person].[BusinessEntityAddress]')
    tables.append('[Person].[BusinessEntityContact]')
    tables.append('[Person].[ContactType]')
    tables.append('[Person].[CountryRegion]')
    tables.append('[Person].[EmailAddress]')
    tables.append('[Person].[Password]')
    tables.append('[Person].[Person]')
    tables.append('[Person].[PersonPhone]')
    tables.append('[Person].[PhoneNumberType]')
    tables.append('[Person].[StateProvince]')
    tables.append('[Production].[BillOfMaterials]')
    tables.append('[Production].[Culture]')
    tables.append('[Production].[Document]')
    tables.append('[Production].[Illustration]')
    tables.append('[Production].[Location]')
    tables.append('[Production].[Product]')
    tables.append('[Production].[ProductCategory]')
    tables.append('[Production].[ProductCostHistory]')
    tables.append('[Production].[ProductDescription]')
    tables.append('[Production].[ProductDocument]')
    tables.append('[Production].[ProductInventory]')
    tables.append('[Production].[ProductListPriceHistory]')
    tables.append('[Production].[ProductModel]')
    tables.append('[Production].[ProductModelIllustration]')
    tables.append('[Production].[ProductModelProductDescriptionCulture]')
    tables.append('[Production].[ProductPhoto]')
    tables.append('[Production].[ProductProductPhoto]')
    tables.append('[Production].[ProductReview]')
    tables.append('[Production].[ProductSubcategory]')
    tables.append('[Production].[ScrapReason]')
    tables.append('[Production].[TransactionHistory]')
    tables.append('[Production].[TransactionHistoryArchive]')
    tables.append('[Production].[UnitMeasure]')
    tables.append('[Production].[WorkOrder]')
    tables.append('[Production].[WorkOrderRouting]')
    tables.append('[Purchasing].[ProductVendor]')
    tables.append('[Purchasing].[PurchaseOrderDetail]')
    tables.append('[Purchasing].[PurchaseOrderHeader]')
    tables.append('[Purchasing].[ShipMethod]')
    tables.append('[Purchasing].[Vendor]')
    tables.append('[Sales].[CountryRegionCurrency]')
    tables.append('[Sales].[CreditCard]')
    tables.append('[Sales].[Currency]')
    tables.append('[Sales].[CurrencyRate]')
    tables.append('[Sales].[Customer]')
    tables.append('[Sales].[PersonCreditCard]')
    tables.append('[Sales].[SalesOrderDetail]')
    tables.append('[Sales].[SalesOrderHeader]')
    tables.append('[Sales].[SalesOrderHeaderSalesReason]')
    tables.append('[Sales].[SalesPerson]')
    tables.append('[Sales].[SalesPersonQuotaHistory]')
    tables.append('[Sales].[SalesReason]')
    tables.append('[Sales].[SalesTaxRate]')
    tables.append('[Sales].[SalesTerritory]')
    tables.append('[Sales].[SalesTerritoryHistory]')
    tables.append('[Sales].[ShoppingCartItem]')
    tables.append('[Sales].[SpecialOffer]')
    tables.append('[Sales].[SpecialOfferProduct]')
    tables.append('[Sales].[Store]')
    # Convert array of table names to string
    selected_tables = str(tables).replace("'", '\"')
else:
    selected_tables = 'all'

print(f"Tables to Index: {selected_tables}")

Tables to Index: all


## Create client for AzureML Workspace

The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.



Get `credential` to create `MLClient`.

In [32]:
from azure.identity import DefaultAzureCredential, ClientSecretCredential, EnvironmentCredential, AzureCliCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

# Create ClientSecretCredential as default credential
# Service Principal (Application Client) must have Contributor role on the Azure ML Workspace 
try:
    ## Set expected environment variables
    os.environ['AZURE_TENANT_ID'] = tenant_id
    os.environ['AZURE_CLIENT_ID'] = application_client_id
    os.environ['AZURE_CLIENT_SECRET'] = application_client_secret
    os.environ['AZURE_AUTHORITY_HOST'] = 'https://login.microsoftonline.com'
    credential = ClientSecretCredential(tenant_id=tenant_id, client_id=application_client_id, client_secret=application_client_secret)
except Exception as ex:
    print(ex)
    print('Try DefaultAzureCredential creation')
    # Fall back to DefaultAzureCredential, if ClientSecretCredential does not work
    # DefaultAzureCredential will look for credientials sequentially see docs at
    # https://learn.microsoft.com/en-us/dotnet/api/azure.identity.defaultazurecredential
    try:
        credential = DefaultAzureCredential()
    except Exception as ex:
        print(ex)

credential.get_token("https://management.azure.com/.default")


AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL21hbmFnZW1lbnQuYXp1cmUuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNzJmOTg4YmYtODZmMS00MWFmLTkxYWItMmQ3Y2QwMTFkYjQ3LyIsImlhdCI6MTY5NDQ2NTE4OCwibmJmIjoxNjk0NDY1MTg4LCJleHAiOjE2OTQ1NTE4ODgsImFpbyI6IkUyRmdZUGlhYnRPc0pHcFRMTWcvUWNSUTRPUlBBQT09IiwiYXBwaWQiOiJiZDhlNDU3Ni0zYjA2LTQ5MTgtYmU3ZC0wYjZhODE3NTA5ZWYiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC83MmY5ODhiZi04NmYxLTQxYWYtOTFhYi0yZDdjZDAxMWRiNDcvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI4Y2Q2OWM1Yy04YzU4LTQ5MTgtOTk2Zi1mOTI0ZmFlN2M4NmMiLCJyaCI6IjAuQVJvQXY0ajVjdkdHcjBHUnF5MTgwQkhiUjBaSWYza0F1dGRQdWtQYXdmajJNQk1hQUFBLiIsInN1YiI6IjhjZDY5YzVjLThjNTgtNDkxOC05OTZmLWY5MjRmYWU3Yzg2YyIsInRpZCI6IjcyZjk4OGJmLTg2ZjEtNDFhZi05MWFiLTJkN2NkMDExZGI0NyIsInV0aSI6IldKOEpacDVZSFUyNnNsTXpLM2pJQUEiLCJ2ZXIiOiIxLjAiLCJ4bXNfdGNkdCI6MTI4OTI0MTU0N30.LcazQQbSGkfeIzW_XptWj_YSDoqZlC7bWmRQ

Create `MLClient` to interact with AzureML

In [33]:
try:
    ml_client= MLClient(credential=credential, subscription_id=subscription_id, resource_group_name=resource_group_name, workspace_name=workspace_name)
except Exception as ex:
    raise Exception("Failed to create MLClient from config file. Please verify AzureML Workspace details and update Execution Variables above.") from ex

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

MLClient(credential=<azure.identity._credentials.client_secret.ClientSecretCredential object at 0x0000022D19261F90>,
         subscription_id=8878a446-3d3e-44c2-bae5-09fd1d17e6d6,
         resource_group_name=common-ai-dev,
         workspace_name=common-ai-dev-ml)


## Azure OpenAI

We recommend using gpt-35-turbo model to get good quality QAs. [Follow these instructions](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) to setup an Azure OpenAI Instance and deploy the model. Once you have the model deployed in AOAI you can specify your Model name and Deployment name below.

We will use the automatically created `Default_AzureOpenAI` connection, change `aoai_connection_name` to use your own.

In [34]:
from azureml.rag.utils.connections import get_connection_by_name_v2

try:
    # Get the Azure OpenAI Connection
    aoai_connection = get_connection_by_name_v2(ws, aoai_connection_name)
    # Get the Azure OpenAI connection id
    aoai_connection_id = aoai_connection["id"]
    # Print Azure OpenAI connection info
    print(f"Azure OpenAI connection: \n{json.dumps(aoai_connection, indent=4)}")
    
except Exception as ex:
    print(f'Exception: {ex}')
    print(f'Unable to create a connection to the Azure OpenAI resource named: {aoai_connection_name}')


Azure OpenAI connection: 
{
    "tags": null,
    "location": null,
    "id": "/subscriptions/8878a446-3d3e-44c2-bae5-09fd1d17e6d6/resourceGroups/common-ai-dev/providers/Microsoft.MachineLearningServices/workspaces/common-ai-dev-ml/connections/Default_AzureOpenAI",
    "name": "Default_AzureOpenAI",
    "type": "Microsoft.MachineLearningServices/workspaces/connections",
    "properties": {
        "authType": "ApiKey",
        "credentials": {
            "key": "f7f4b78604f04f1b9851c94af756adde"
        },
        "category": "AzureOpenAI",
        "expiryTime": null,
        "target": "https://common-ai-dev-ml-aoai.openai.azure.com/",
        "createdByWorkspaceArmId": null,
        "isSharedToAll": true,
        "sharedUserList": [],
        "metadata": {
            "ApiVersion": "2023-03-15-preview",
            "ApiType": "Azure",
            "ProvisioningState": "Succeeded",
            "ResourceId": "/subscriptions/8878a446-3d3e-44c2-bae5-09fd1d17e6d6/resourceGroups/common-ai-d

Now that the Workspace has a connection to Azure Open AI ensure the **embedding** model has been deployed (recommendation is `text-embedding-ada-002`)

This cell will fail if there is not deployment for the embeddings model, [follow these instructions](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#deploy-a-model) to deploy a model with Azure OpenAI.

In [35]:
from azureml.rag.utils.deployment import infer_deployment

try:
    aoai_embedding_deployment_name = infer_deployment(aoai_connection, aoai_embedding_model_name)
    print(f"Deployment name in AOAI workspace for model '{aoai_embedding_model_name}' is '{aoai_embedding_deployment_name}'")
except Exception as ex:
    print(f"Exception: {ex}")
    print(f"Deployment name in AOAI workspace for model '{aoai_embedding_model_name}' is not found.")
    print(f"Please create a deployment for this model by following the deploy instructions on the resource page for '{aoai_connection['properties']['target']}' in Azure Portal.")

Deployment name in AOAI workspace for model 'text-embedding-ada-002' is 'text-embedding-ada-002'


Now that the Workspace has a connection to Azure Open AI ensure the **completion** model has been deployed (recommendation is `gpt-35-turbo`)

The following cell will fail if a **completion** model is not deployed, [follow these instructions](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#deploy-a-model) to deploy a **completion** model with Azure OpenAI.

In [36]:
from azureml.rag.utils.deployment import infer_deployment

try:
    aoai_completion_deployment_name = infer_deployment(aoai_connection, aoai_completion_model_name)
    print(f"Deployment name in AOAI workspace for model '{aoai_completion_model_name}' is '{aoai_completion_deployment_name}'")
except Exception as ex:
    print(f"Exception: {ex}")
    print(f"Deployment name in AOAI workspace for model '{aoai_completion_model_name}' is not found.")
    print(f"Please create a deployment for this model by following the deploy instructions on the resource page for '{aoai_connection['properties']['target']}' in Azure Portal.")

# Create LLM completion config in URI form which the AzureML embeddings components expect as input.
llm_completion_config = f'{{"type":"azure_open_ai","model_name":"{aoai_completion_model_name}","deployment_name":"{aoai_completion_deployment_name}","temperature":0,"max_tokens":"1500"}}'

Deployment name in AOAI workspace for model 'gpt-35-turbo' is 'gpt-35-turbo'


### Setup Pipeline Job

The Components are published to a [Registry](https://learn.microsoft.com/azure/machine-learning/how-to-manage-registries?view=azureml-api-2&tabs=cli), `azureml`, which should have access to by default, it can be accessed from any Workspace.
In the below cell we get the Component Definitions from the `azureml` registry.

In [37]:
ml_registry = MLClient(credential=credential, registry_name="azureml")

db_copilot_component = ml_registry.components.get("llm_ingest_db_to_faiss", label="latest")

print(db_copilot_component)

$schema: https://azuremlschemas.azureedge.net/latest/pipelineComponent.schema.json
name: llm_ingest_db_to_faiss
version: 0.0.21
display_name: LLM - SQL Datastore to FAISS Pipeline
description: Single job pipeline to chunk data from AzureML sql data store, and create
  FAISS embeddings index
type: pipeline
inputs:
  db_datastore:
    type: string
    optional: false
    description: database datastore uri in the format of 'azureml://datastores/{datastore_name}'
  embeddings_model:
    type: string
    optional: false
    description: The model used to generate embeddings. 'azure_open_ai://endpoint/{endpoint_name}/deployment/{deployment_name}/model/{model_name}'
  chat_aoai_deployment_name:
    type: string
    optional: true
    description: The name of the chat AOAI deployment
  embedding_aoai_deployment_name:
    type: string
    optional: false
    description: The name of the embedding AOAI deployment
  embeddings_dataset_name:
    type: string
    optional: false
    description: T

In [38]:
# Create the pipeline
from azure.ai.ml.dsl import pipeline
@pipeline(
    name=f"db_copilot_vector_pipeline_faiss",
    default_compute=default_compute
)
def db_copilot_vector_pipeline_faiss(
    aoai_connection: str,
    db_datastore: str,
    embeddings_model: str,
    chat_aoai_deployment_name: str,
    embedding_aoai_deployment_name: str,
    mlindex_dataset_name: str,
    selected_tables: str = None,
    max_sampling_rows: int = 3,
):
    db_copilot_component(
        db_datastore=db_datastore,
        embeddings_model=embeddings_model,
        chat_aoai_deployment_name=chat_aoai_deployment_name,
        embedding_aoai_deployment_name=embedding_aoai_deployment_name,
        embeddings_dataset_name=mlindex_dataset_name,
        embedding_connection=aoai_connection,
        llm_connection=aoai_connection,
        selected_tables=selected_tables,
        max_sampling_rows=max_sampling_rows,
    )
    return {}


In [39]:
# Create pipeline job
pipeline_job = db_copilot_vector_pipeline_faiss(
    aoai_connection=aoai_connection_id,
    db_datastore=f"azureml://datastores/{datastore_name}",
    embeddings_model=f"azure_open_ai://deployment/{aoai_embedding_deployment_name}/model/{aoai_embedding_model_name}",
    chat_aoai_deployment_name=aoai_completion_deployment_name,
    embedding_aoai_deployment_name=aoai_embedding_deployment_name,
    mlindex_dataset_name=data_asset_name,
    selected_tables=selected_tables,
    max_sampling_rows=3,
)


In [40]:
# These are added so that in progress index generations can be listed in UI, this tagging is done automatically by UI.
pipeline_job.properties["azureml.mlIndexAssetName"] = data_asset_name
pipeline_job.properties["azureml.mlIndexAssetKind"] = "faiss"
pipeline_job.properties["azureml.mlIndexAssetSource"] = "Database"

In [41]:
# Submit pipeline job
running_pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name=str.lower(f"{datastore_name}_{datastore_scope}_dbcopilot_pipeline")
)
running_pipeline_job

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Experiment,Name,Type,Status,Details Page
adventureworks2022_all_dbcopilot_pipeline,bright_cumin_384gpfjhy6,pipeline,Preparing,Link to Azure Machine Learning studio


In [42]:
ml_client.jobs.stream(running_pipeline_job.name)

RunId: bright_cumin_384gpfjhy6
Web View: https://ml.azure.com/runs/bright_cumin_384gpfjhy6?wsid=/subscriptions/8878a446-3d3e-44c2-bae5-09fd1d17e6d6/resourcegroups/common-ai-dev/workspaces/common-ai-dev-ml

Streaming logs/azureml/executionlogs.txt

[2023-09-11 20:51:44Z] Submitting 1 runs, first five are: f36318d1:d37659bf-2f2a-4c1f-8772-d68af2818dcb
[2023-09-11 21:00:24Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: bright_cumin_384gpfjhy6
Web View: https://ml.azure.com/runs/bright_cumin_384gpfjhy6?wsid=/subscriptions/8878a446-3d3e-44c2-bae5-09fd1d17e6d6/resourcegroups/common-ai-dev/workspaces/common-ai-dev-ml


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. Failed nodes: /llm_ingest_db_to_faiss/db_meta_loading_generator. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus",
    "location": "eastus",
    "time": "2023-09-11T21:00:24.342952Z",
    "component_name": ""
} 

## Use DBCopilot with Promptflow
After the pipeline complete, it will create a promptflow which could be used to chat with the db.